In [1]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 43.4 MB/s eta 0:00:00


In [2]:
!pip install jupyter-dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import dash
from dash import dcc, html, Output, Input, State, ctx
from dash import dash_table
import plotly.graph_objs as go
from jupyter_dash import JupyterDash

# --- Load data ---
data = pd.read_csv('/content/synced_final_with_initial_and_process_data (1).csv', low_memory=False)

# --- Scene boundaries ---
x_min, x_max = data['X_world'].min(), data['X_world'].max()
y_min, y_max = data['Y_world'].min(), data['Y_world'].max()
z_min, z_max = data['Z_world'].min(), data['Z_world'].max()

center_x, center_y, center_z = data['X_world'].mean(), data['Y_world'].mean(), data['Z_world'].mean()
range_x = (x_max - x_min) * 0.4
range_y = (y_max - y_min) * 0.15
range_z = (z_max - z_min) * 0.1

# --- Process graph data ---
process_data = data[['Time_ProcessData', 'WFS_Process_Data', 'I_Process_Data', 'U_Process_Data', 'Layer']].dropna()

# --- Initialize Dash ---
app = dash.Dash(__name__, suppress_callback_exceptions=True)

# --- Application layout ---
app.layout = html.Div([
    # Left block - Filters
    html.Div([
        html.H4("Filters:", style={'marginBottom': '15px'}),

        html.Label("Minimum Intensity:", style={'fontSize': '14px', 'marginBottom': '5px'}),
        dcc.Slider(
            id='intensity-slider',
            min=data['Intensity'].min(),
            max=data['Intensity'].max(),
            value=20,
            tooltip={"placement": "bottom"},
            className="thin-track"
        ),
        html.Br(),

        html.Label("X Range:", style={'fontSize': '14px', 'marginBottom': '5px'}),
        dcc.RangeSlider(
            id='x-range',
            min=x_min,
            max=x_max,
            value=[x_min, x_max],
            className="thin-track"
        ),
        html.Br(),

        html.Label("Y Range:", style={'fontSize': '14px', 'marginBottom': '5px'}),
        dcc.RangeSlider(
            id='y-range',
            min=y_min,
            max=y_max,
            value=[y_min, y_max],
            className="thin-track"
        ),
        html.Br(),

        html.Label("Z Range:", style={'fontSize': '14px', 'marginBottom': '5px'}),
        dcc.RangeSlider(
            id='z-range',
            min=z_min,
            max=z_max,
            value=[z_min, z_max],
            className="thin-track"
        ),
        html.Br(),

        html.Label("Select Layers:", style={'fontSize': '14px', 'marginBottom': '5px'}),
        dcc.Dropdown(
            id='layer-selector',
            options=[{'label': str(l), 'value': l} for l in sorted(data['Layer'].unique())],
            value=sorted(data['Layer'].unique()),
            multi=True,
            style={'marginBottom': '10px'}
        ),

        html.Div([
            html.Button('Show All Layers', id='show-all', n_clicks=0, style={'marginRight': '5px', 'marginBottom': '5px'}),
            html.Button('Hide All Layers', id='hide-all', n_clicks=0, style={'marginBottom': '5px'}),
        ], style={'display': 'flex', 'flexWrap': 'wrap'}),

        html.Div([
            html.Button('Fill Table', id='fill-table', n_clicks=0, style={'marginRight': '5px', 'marginBottom': '5px'}),
            html.Button('Clear Table', id='clear-table', n_clicks=0, style={'marginBottom': '5px'}),
        ], style={'display': 'flex', 'flexWrap': 'wrap'}),

        html.Hr(),

        html.H4("Cropping:", style={'marginBottom': '10px'}),
        html.Label("X Range:", style={'fontSize': '14px', 'marginBottom': '5px'}),
        dcc.RangeSlider(
            id='crop-x',
            min=x_min,
            max=x_max,
            value=[center_x - range_x, center_x + range_x],
            className="thin-track"
        ),
        html.Br(),

        html.Label("Y Range:", style={'fontSize': '14px', 'marginBottom': '5px'}),
        dcc.RangeSlider(
            id='crop-y',
            min=y_min,
            max=y_max,
            value=[center_y - range_y, center_y + range_y],
            className="thin-track"
        ),
        html.Br(),

        html.Label("Z Range:", style={'fontSize': '14px', 'marginBottom': '5px'}),
        dcc.RangeSlider(
            id='crop-z',
            min=z_min,
            max=z_max,
            value=[center_z - range_z, center_z + range_z],
            className="thin-track"
        ),
        html.Br(),

        html.Button('Reset Crop', id='reset-crop', n_clicks=0, style={'marginBottom': '10px'}),
        html.Hr(),

        html.H4("Points:", style={'marginBottom': '10px'}),
        dcc.Slider(
            id='max-points-slider',
            min=10000,
            max=1000000,
            value=50000,
            tooltip={"placement": "bottom"},
            className="thin-track"
        ),
        html.Br(),

        html.Button('Show All Points', id='show-all-points', n_clicks=0, style={'marginBottom': '10px'}),
    ], style={
        'width': '20%',
        'padding': '10px',
        'overflowY': 'auto',
        'height': '100vh',
        'display': 'inline-block',
        'verticalAlign': 'top'
    }),

    # Central part (Points + Table + Process Graph)
    html.Div([
        html.Div([
            html.Div([
                html.Div(id='point-counts', style={'marginBottom': '5px', 'fontWeight': 'bold'}),
                dcc.Graph(id='3d-scatter', style={'height': '45vh', 'width': '100%'})
            ], style={'width': '60%', 'display': 'inline-block', 'verticalAlign': 'top'}),

            html.Div([
                html.H4("Initial Data", style={'marginBottom': '5px'}),
                html.Div("Selected Layers:", style={'fontSize': '14px', 'fontWeight': 'bold', 'marginBottom': '10px'}),
                dash_table.DataTable(
                    id='table',
                    columns=[
                        {"name": "Layer", "id": "Layer"},
                        {"name": "Initial_Job", "id": "Initial_Job"},
                        {"name": "Initial_TS", "id": "Initial_TS"}
                    ],
                    data=[],
                    page_size=20,
                    style_table={'overflowY': 'auto', 'height': '45vh', 'marginTop': '5px'},
                    style_cell={'textAlign': 'left', 'fontSize': '12px'}
                )
            ], style={'width': '40%', 'padding': '10px', 'verticalAlign': 'top', 'display': 'inline-block'})
        ], style={'display': 'flex', 'width': '100%'}),

        # Wide Process Graph
        html.Div([
            dcc.Graph(id='process-graph', style={'height': '50vh', 'width': '100%'})
        ], style={'width': '100%', 'marginTop': '10px'})
    ], style={
        'width': '80%',
        'display': 'inline-block',
        'verticalAlign': 'top',
        'height': '100vh'
    })
], style={'display': 'flex', 'flexDirection': 'row', 'height': '100vh'})


# --- Callbacks ---
@app.callback(
    Output('3d-scatter', 'figure'),
    Output('point-counts', 'children'),
    Input('intensity-slider', 'value'),
    Input('x-range', 'value'),
    Input('y-range', 'value'),
    Input('z-range', 'value'),
    Input('layer-selector', 'value'),
    Input('crop-x', 'value'),
    Input('crop-y', 'value'),
    Input('crop-z', 'value'),
    Input('max-points-slider', 'value'),
    Input('show-all-points', 'n_clicks')
)
def update_graph(intensity, x_range, y_range, z_range, layers, crop_x, crop_y, crop_z, max_points, show_all_points):
    filtered = data[
        (data['Intensity'] >= intensity) &
        (data['X_world'].between(x_range[0], x_range[1])) &
        (data['Y_world'].between(y_range[0], y_range[1])) &
        (data['Z_world'].between(z_range[0], z_range[1]))
    ]
    cropped = filtered[
        (filtered['X_world'].between(crop_x[0], crop_x[1])) &
        (filtered['Y_world'].between(crop_y[0], crop_y[1])) &
        (filtered['Z_world'].between(crop_z[0], crop_z[1]))
    ]
    visible = cropped[cropped['Layer'].isin(layers)]
    sampled = visible if ctx.triggered_id == 'show-all-points' else visible.sample(min(len(visible), max_points))

    fig = go.Figure(data=[
        go.Scatter3d(
            x=sampled['X_world'],
            y=sampled['Y_world'],
            z=sampled['Z_world'],
            mode='markers',
            marker=dict(size=2, color=pd.factorize(sampled['Layer'])[0], colorscale='Viridis'),
            customdata=sampled[['Layer', 'Initial_Job', 'Initial_TS']],
            hovertemplate="Layer=%{customdata[0]}<br>Job=%{customdata[1]}<br>TS=%{customdata[2]}",
            name=''
        )
    ])
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0), height=400,
                      scene=dict(aspectmode='cube', aspectratio=dict(x=1, y=1, z=1),
                                 xaxis=dict(range=[x_min, x_max]),
                                 yaxis=dict(range=[y_min, y_max]),
                                 zaxis=dict(range=[z_min, z_max])))
    return fig, f"Total Points: {len(filtered)} | After Crop: {len(cropped)} | Visible: {len(sampled)}"

# --- Callback for updating the process data graph ---
@app.callback(
    Output('process-graph', 'figure'),
    Input('intensity-slider', 'value'),
    Input('x-range', 'value'),
    Input('y-range', 'value'),
    Input('z-range', 'value'),
    Input('layer-selector', 'value'),
    Input('crop-x', 'value'),
    Input('crop-y', 'value'),
    Input('crop-z', 'value')
)
def update_process_graph(intensity, x_range, y_range, z_range, layers, crop_x, crop_y, crop_z):
    filtered = process_data[
        (data['X_world'].between(x_range[0], x_range[1])) &
        (data['Y_world'].between(y_range[0], y_range[1])) &
        (data['Z_world'].between(z_range[0], z_range[1])) &
        (data['Layer'].isin(layers))
    ]

    fig = go.Figure()

    for column, name, color in [
        ('WFS_Process_Data', 'WFS', 'blue'),
        ('I_Process_Data', 'Current (I)', 'red'),
        ('U_Process_Data', 'Voltage (U)', 'green')
    ]:
        fig.add_trace(go.Scatter(
            x=filtered['Time_ProcessData'],
            y=filtered[column],
            mode='lines',
            name=name,
            line=dict(color=color),
            customdata=filtered[['Layer']],
            hovertemplate='Layer: %{customdata[0]}<br>' + name + ': %{y:.2f}<extra></extra>'
        ))

    fig.update_layout(
        title='Process Data',
        xaxis_title='Time',
        yaxis_title='Value',
        height=500,
        margin=dict(l=10, r=10, t=30, b=10),
        xaxis=dict(
            tickangle=90
        ),
        hovermode='x unified',
        legend_title='Parameters'
    )

    return fig

# --- Callback for adding a layer to the table ---
@app.callback(
    Output('table', 'data', allow_duplicate=True),
    Input('3d-scatter', 'clickData'),
    State('table', 'data'),
    prevent_initial_call='initial_duplicate'
)
def add_layer(clickData, current_table):
    if not clickData:
        raise dash.exceptions.PreventUpdate
    clicked_layer = clickData['points'][0]['customdata'][0]
    if any(row['Layer'] == clicked_layer for row in current_table):
        return current_table
    layer_info = data[data['Layer'] == clicked_layer][['Layer', 'Initial_Job', 'Initial_TS']].drop_duplicates('Layer')
    return current_table + layer_info.to_dict('records')

# --- Callback for managing layers selection ---
@app.callback(
    Output('layer-selector', 'value', allow_duplicate=True),
    Input('show-all', 'n_clicks'),
    Input('hide-all', 'n_clicks'),
    prevent_initial_call='initial_duplicate'
)
def manage_layers(show_all, hide_all):
    triggered_id = ctx.triggered_id
    if triggered_id == 'show-all':
        return sorted(data['Layer'].unique())
    elif triggered_id == 'hide-all':
        return []

# --- Callback for resetting crop ranges ---
@app.callback(
    Output('crop-x', 'value'),
    Output('crop-y', 'value'),
    Output('crop-z', 'value'),
    Input('reset-crop', 'n_clicks')
)
def reset_crop(n):
    if n > 0:
        return [x_min, x_max], [y_min, y_max], [z_min, z_max]
    raise dash.exceptions.PreventUpdate

# --- Callback for managing the table (fill/clear) ---
@app.callback(
    Output('table', 'data', allow_duplicate=True),
    Input('fill-table', 'n_clicks'),
    Input('clear-table', 'n_clicks'),
    prevent_initial_call='initial_duplicate'
)
def manage_table(fill, clear):
    triggered_id = ctx.triggered_id
    if triggered_id == 'fill-table':
        return data[['Layer', 'Initial_Job', 'Initial_TS']].drop_duplicates('Layer').to_dict('records')
    elif triggered_id == 'clear-table':
        return []

# --- Run ---
if __name__ == "__main__":
    app.run(debug=True)
